In [1]:
from typing import Dict
from l5kit.data.map_api import MapAPI
# from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym
from l5kit.environment.envs.l5_env2 import GymStepOutput, SimulationConfigGym, L5Env2
from l5kit.environment.envs.l5_env import GymStepOutput, SimulationConfigGym, L5Env
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.logger import pretty_print
import ray.rllib.algorithms.ppo as ppo
from ray import tune
from ray.rllib.models import ModelCatalog
# from wrapper import L5EnvWrapper
from testWrapper import L5EnvWrapper
from customModel import TFGNNCNN, TorchGNCNN, TorchAttentionModel, TorchAttentionModel2
import os

from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from tempfile import gettempdir

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDatasetVectorized
from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel
from l5kit.planning.vectorized.open_loop_model import VectorizedModel
from l5kit.vectorization.vectorizer_builder import build_vectorizer


In [2]:
# get environment config
env_config_path = '/home/pronton/rl/l5kit/examples/RL/gym_vectorizer_config.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
###############

In [4]:
def rollout_episode(env, idx = 0):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.set_reset_id(idx)
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    while True:
        # action = np.array(env.action_space.sample())
        obs, _, done, info = env.step(np.array(env.action_space.sample()))
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out

sim_outs =[]
for i in range(1):
    sim_outs.append(rollout_episode(rollout_env, i))

/home/pronton/.virtualenvs/l5kit-nr_j5XnQ/lib/python3.8/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [ ]:
def rollout_episode(env, idx = 0):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.set_reset_id(idx)
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    while True:
        # action = np.array(env.action_space.sample())
        obs, _, done, info = env.step(np.array(env.action_space.sample()))
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out

In [4]:
env_config_path = '/home/pronton/rl/l5kit/examples/RL/gym_vectorizer_config.yaml'
# env_config_path = '/home/pronton/rl/l5kit/examples/RL/gym_config.yaml'
dmg = LocalDataManager(None)
cfg = load_config_data(env_config_path)
train_sim_cfg = SimulationConfigGym()

# env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# # tune.register_env("L5-CLE-V0", lambda config: L5Env(**env_kwargs))
# env = L5Env2(**env_kwargs)
# print(env.observation_space)
# print(env._get_obs(0,True))
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': rollout_sim_cfg,  'train': False, 'return_info': True}
# rollout_env = L5Env2(**env_kwargs)
rollout_env = L5Env2(**env_kwargs)



/home/pronton/rl/l5kit/l5kit/l5kit/data/map_api.py:399: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lanes_bounds = np.empty((0, 2, 2), dtype=np.float)  # [(X_MIN, Y_MIN), (X_MAX, Y_MAX)]
/home/pronton/rl/l5kit/l5kit/l5kit/data/zarr_dataset.py:213: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  dataset = ChunkedDataset("")
/home/pronton/rl/l5kit/l5kit/l5kit/simulation/utils.py:107: RuntimeWarning: zarr dataset path should end with .zarr (for now). Open will fail for this dataset!
  new_dataset = ChunkedDataset("")
/home/pronton/rl/l5kit/l5kit/l5kit/sampling/agent_sampling_vectorized.py:129: DeprecationWarni

In [ ]:
from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle, simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
sim_outs =[]
for i in range(1):
    sim_outs.append(rollout_episode(rollout_env, i))

# rollout_env = L5Env(**env_kwargs)
# might change with different rasterizer
# map_API = rollout_env.dataset.rasterizer.sem_rast.mapAPI
mapAPI = MapAPI.from_cfg(dmg, cfg)

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs, mapAPI)

In [3]:
train_eps_length = 10
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg}
tune.register_env("L5-CLE-V2", lambda config: L5Env2(**env_kwargs))
ray.init(num_cpus=1, ignore_reinit_error=True, log_to_driver=False)
ModelCatalog.register_custom_model( "TorchSeparatedAttentionModel", TorchAttentionModel)
ModelCatalog.register_custom_model( "TorchSharedAttentionModel", TorchAttentionModel2)
ModelCatalog.register_custom_model("GN_CNN_torch_model", TorchGNCNN)

# Create the Trainer.


2023-02-26 22:26:39,390	INFO worker.py:1538 -- Started a local Ray instance.


## L5env1

In [ ]:
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': False, 'sim_cfg': train_sim_cfg}

tune.register_env("L5-CLE-V1", lambda config: L5EnvWrapper(L5Env(**env_kwargs)))
ray.init(num_cpus=1, ignore_reinit_error=True, log_to_driver=False)
ModelCatalog.register_custom_model(
        "tcnn", TorchGNCNN
    )
# Create the Trainer.
algo = ppo.PPO(
        env="L5-CLE-V1",
        config={
            'num_worker': 1,
            'disable_env_checking':True,
            "framework": "torch",
            # "model": {
            #     "custom_model": "GN_CNN_torch_model",
            #     "custom_model_config": {'feature_dim':128},
            # },
            "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(2):
    result = algo.train()
    print(pretty_print(result))

In [4]:

train_eps_length = 10
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1

env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env = L5Env2(**env_kwargs)
# env1_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
# env1 = L5Env(**env_kwargs)


## L5env2

In [4]:
algo = ppo.PPO(
        env="L5-CLE-V2",
        config={
            'disable_env_checking':True,
            "framework": "torch",
            'log_level': 'INFO',
            'train_batch_size'
            'train_batch_size': 1,
            'sgd_minibatch_size': 1,
            'num_sgd_iter': 1,
            'seed': 42,
            'batch_mode': 'truncate_episodes',
            # "rollout_fragment_length": 32,
            "model": {
                "custom_model": "TorchSharedAttentionModel",
                # Extra kwargs to be passed to your model's c'tor.
                "custom_model_config": {'cfg':cfg},
            },
            # "output": "/home/pronton/rl/l5kit/examples/RL/notebooks/logs/l5env2-out", 
            # "output_max_file_size": 5000000,
            '_disable_preprocessor_api': True,
        },
    )

for i in range(1):
    result = algo.train()
    print(pretty_print(result))

2023-02-26 22:26:40,015	WARNING algorithm_config.py:488 -- Cannot create PPOConfig from given `config_dict`! Property train_batch_sizetrain_batch_size not supported.
2023-02-26 22:26:40,017	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-V2', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-V2').build()` instead. This will raise an error in the future!


KeyboardInterrupt: 